# Step 1: Real-Time Feature Computation

This notebook is part of a demo showcasing a real-time fraud detection pipeline, utilizing Feldera for feature computation and Hopsworks as the feature store.

![Real-time feature engineering pipeline using Feldera and Hosworks](./architecture.png)


In [ ]:
from feldera import FelderaClient, PipelineBuilder

## Set Hopsworks API KEY

In [ ]:
from ipython_secrets import *

KEY = get_secret("HOPSWORKS_API_KEY")

In [ ]:
# Connect to the Feldera API

# Use Feldera online sandbox
# client = FelderaClient("https://try.feldera.com", api_key = get_secret('FELDERA_API_KEY'))

# Use local Feldera instance
client = FelderaClient("http://localhost:8080")

## Step 1.1. Create Hopsworks feature groups

In [ ]:
import hopsworks
from hsfs import engine
from hsfs.feature import Feature
import json

# Connect to Hopsworks.
project = hopsworks.login(host="c.app.hopsworks.ai", api_key_value=KEY)

kafka_api = project.get_kafka_api()
KAFKA_OUTPUT_TOPICS = [
    "transactions_fraud_streaming_fg_" + str(project.id),
    "transactions_aggs_fraud_streaming_fg_" + str(project.id),
]

fs = project.get_feature_store()

# Create feature groups to store Feldera outputs.

# COMBINED - features that extend credit card transaction records with attributes extracted from the card
# holder's profile, such as their age at the time of the transaction and the number of days until the credit card expires.
combined_fg = fs.get_or_create_feature_group(
    name=KAFKA_OUTPUT_TOPICS[0],
    primary_key=["cc_num"],
    online_enabled=True,
    version=1,
    topic_name=KAFKA_OUTPUT_TOPICS[0],
    event_time="date_time",
    stream=True,
    features=[
        Feature("tid", type="string"),
        Feature("date_time", type="timestamp"),
        Feature("cc_num", type="string"),
        Feature("category", type="string"),
        Feature("amount", type="double"),
        Feature("latitude", type="double"),
        Feature("longitude", type="double"),
        Feature("city", type="string"),
        Feature("country", type="string"),
        Feature("fraud_label", type="int"),
        Feature("age_at_transaction", type="int"),
        Feature("days_until_card_expires", type="int"),
        Feature("cc_expiration_date", type="timestamp"),
    ],
)

try:
    combined_fg.save()
except Exception as e:
    print(e)

if KAFKA_OUTPUT_TOPICS[0] not in [topic.name for topic in kafka_api.get_topics()]:
    kafka_api.create_schema(KAFKA_OUTPUT_TOPICS[0], json.loads(combined_fg.avro_schema))
    kafka_api.create_topic(
        KAFKA_OUTPUT_TOPICS[0], KAFKA_OUTPUT_TOPICS[0], 1, replicas=1, partitions=1
    )

# WINDOWED - frequency of transactions and other metrics in the span of a few hours, modeled as hopping window aggregates.
windowed_fg = fs.get_or_create_feature_group(
    name=str(KAFKA_OUTPUT_TOPICS[1]),
    primary_key=["cc_num"],
    online_enabled=True,
    version=1,
    topic_name=KAFKA_OUTPUT_TOPICS[1],
    event_time="date_time",
    stream=True,
    features=[
        Feature("avg_amt", type="double"),
        Feature("trans", type="bigint"),
        Feature("stddev_amt", type="double"),
        Feature("date_time", type="timestamp"),
        Feature("cc_num", type="string"),
    ],
)

try:
    windowed_fg.save()
except Exception as e:
    print(e)

if KAFKA_OUTPUT_TOPICS[1] not in [topic.name for topic in kafka_api.get_topics()]:
    kafka_api.create_schema(KAFKA_OUTPUT_TOPICS[1], json.loads(windowed_fg.avro_schema))
    kafka_api.create_topic(
        KAFKA_OUTPUT_TOPICS[1], KAFKA_OUTPUT_TOPICS[1], 1, replicas=1, partitions=1
    )

## Step 1.2. Create Feldera pipeline

We build a Feldera pipeline to transform raw transaction and profile data into features. In Feldera, feature groups are modeled as SQL views. Thus, we create a SQL program with two input tables (TRANSACTIONS and PROFILES), and two output views, one for each feature group.

![Feldera pipeline](./feldera_pipeline.png)

In [ ]:
# Create SQL program parameterized by source and sink connnector configurations.
def build_sql(
    transaction_source_config: str, combined_sink_config: str, windowed_sink_config: str
) -> str:
    return f"""
    CREATE TABLE TRANSACTIONS(
        tid STRING,
        date_time TIMESTAMP,
        cc_num STRING,
        category STRING,
        amount DECIMAL(38, 2),
        latitude DOUBLE,
        longitude DOUBLE,
        city STRING,
        country STRING,
        fraud_label INT
    ) WITH (
        'connectors' = '[{transaction_source_config}]'
    );
    
    CREATE TABLE PROFILES(
        cc_num STRING,
        cc_provider STRING,
        cc_type STRING,
        cc_expiration_date STRING,
        name STRING,
        mail STRING,
        birthdate TIMESTAMP,
        age INT,
        city STRING,
        country_of_residence STRING
    );
    
    -- Convert credit card expiration date from MM/YY formatted string to a TIMESTAMP,
    -- so that we can perform computations on it.
    CREATE LOCAL VIEW CC_EXPIRATION as
        SELECT
            cc_num,
            CAST(
                CONCAT(
                    '20',
                    SUBSTRING(
                        cc_expiration_date,
                        4,
                        2
                    ),
                    '-',
                    SUBSTRING(
                        cc_expiration_date,
                        1,
                        2
                    ),
                    '-01 00:00:00'
                ) AS TIMESTAMP
            ) AS cc_expiration_date
        FROM PROFILES;
    
    -- Compute the age of the individual during the transaction, and the number of days until the
    -- credit card expires from `PROFILES` and `TRANSACTIONS` tables.
    CREATE VIEW COMBINED 
    WITH (
        'connectors' = '[{combined_sink_config}]'
    )
    AS
        SELECT
            T1.*,
            T2.cc_expiration_date,
            TIMESTAMPDIFF(YEAR, T3.birthdate, T1.date_time) age_at_transaction,
            TIMESTAMPDIFF(DAY, T1.date_time, T2.cc_expiration_date) days_until_card_expires
        FROM
            TRANSACTIONS T1 JOIN cc_expiration T2
            ON
                T1.cc_num = T2.cc_num
            JOIN PROFILES T3
        ON
            T1.cc_num = T3.cc_num;
    
    -- Create a 4 hour hopping window aggregation from data from transactions table
    CREATE LOCAL VIEW HOP as 
        SELECT * 
        FROM TABLE(HOP(TABLE TRANSACTIONS, DESCRIPTOR(date_time), INTERVAL 4 HOURS, INTERVAL 1 HOURS));
    
    -- Compute aggregates from it
    CREATE LOCAL VIEW AGG as
        SELECT
            AVG(amount) AS avg_amt,
            STDDEV(amount) as stddev_amt,
            COUNT(cc_num) as trans,
            ARRAY_AGG(date_time) as moments,
            cc_num
        FROM hop
        GROUP BY cc_num, window_start;
    
    -- Final output view
    CREATE VIEW WINDOWED
    WITH (
        'connectors' = '[{windowed_sink_config}]'
    )
    AS
        SELECT
            avg_amt,
            trans,
            COALESCE(stddev_amt, 0) as stddev_amt,
            date_time,
            cc_num
        FROM agg CROSS JOIN UNNEST(moments) as date_time;
    """

### Connect Kafka sources and sinks

We use the Kafka topic created during the data prep step as the input for the TRANSACTIONS table. The output views are also connected to the Hopsworks feature store via Kafka. Hopsworks ingests data from Kafka using the Avro format, so we configure Feldera Kafka connectors with Avro schemas generated by Hopsworks for each feature group.

In [ ]:
# Get Hopsworks public Kafka servers.
kafka_config = engine.get_instance()._get_kafka_config(fs.id, {})

KAFKA_INPUT_TOPIC = "transactions_topic_" + str(project.id)

# Connect the Kafka topic created during data prep to the TRANSACTIONS table.
transaction_source_config = json.dumps(
    {
        "transport": {
            "name": "kafka_input",
            "config": kafka_config
            | {"topics": [KAFKA_INPUT_TOPIC], "auto.offset.reset": "earliest"},
        },
        "format": {"name": "json", "config": {"update_format": "raw", "array": False}},
    }
)


def create_sink_config(kafka_config: dict, fg, project_id):
    return kafka_config | {
        "topic": fg.topic_name,
        "auto.offset.reset": "earliest",
        "headers": [
            {
                "key": "projectId",
                "value": str(project_id),
            },
            {
                "key": "featureGroupId",
                "value": str(fg.id),
            },
            {
                "key": "subjectId",
                "value": str(fg.subject["id"]),
            },
        ],
    }


# Set the output format to use the avro schema from the feature group.
combined_sink_config = json.dumps(
    {
        "transport": {
            "name": "kafka_output",
            "config": create_sink_config(kafka_config, combined_fg, project.id),
        },
        "format": {
            "name": "avro",
            "config": {"schema": combined_fg.avro_schema, "skip_schema_id": True},
        },
    }
)

windowed_sink_config = json.dumps(
    {
        "transport": {
            "name": "kafka_output",
            "config": create_sink_config(kafka_config, windowed_fg, project.id),
        },
        "format": {
            "name": "avro",
            "config": {"schema": windowed_fg.avro_schema, "skip_schema_id": True},
        },
    }
)

sql = build_sql(transaction_source_config, combined_sink_config, windowed_sink_config)
pipeline = PipelineBuilder(client, name="hopsworks_kafka", sql=sql).create_or_replace()

## Step 1.3. Run the pipeline

In [ ]:
# Start the Feldera pipeline.
pipeline.start()

# Read profile data from the feature store and write it to the `PROFILE` table.
profile_fg = fs.get_or_create_feature_group(name="profile", version=1)

profile_df = profile_fg.read()

pipeline.input_pandas("PROFILES", profile_df)

In [ ]:
# Run the pipeline for 60s.
import time

time.sleep(60)

pipeline.stop(force=True)

## Set a materialization job for the feature group in Hopsworks

In [ ]:
import datetime

# materialize every 10 minutes
combined_fg.materialization_job.schedule(
    cron_expression="0 /10 * ? * * *",
    start_time=datetime.datetime.now(tz=datetime.timezone.utc),
)
windowed_fg.materialization_job.schedule(
    cron_expression="0 /10 * ? * * *",
    start_time=datetime.datetime.now(tz=datetime.timezone.utc),
)

In [ ]:
combined_fg.materialization_job.run()
windowed_fg.materialization_job.run()

# Great Expectations

In [ ]:
import great_expectations as ge
from great_expectations.core import ExpectationConfiguration

# Set the expectation suite name to "transactions_suite"
expectation_suite_transactions = ge.core.ExpectationSuite(
    expectation_suite_name="transactions_suite"
)

In [ ]:
# Check binary fraud_label column to be in set [0,1]
expectation_suite_transactions.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_distinct_values_to_be_in_set",
        kwargs={
            "column": "fraud_label",
            "value_set": [0, 1],
        },
    )
)

# Check amount column to be not negative
expectation_suite_transactions.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column": "amount",
            "min_value": 0.0,
        },
    )
)

# Loop through specified columns ('tid', 'date_time', 'cc_num') and add expectations for null values
for column in ["tid", "date_time", "cc_num"]:
    expectation_suite_transactions.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_values_to_be_null",
            kwargs={
                "column": column,
                "mostly": 0.0,
            },
        )
    )

In [ ]:
# update the feature group to store this expectation suite
combined_fg.save_expectation_suite(expectation_suite_transactions)

In [ ]:
# Update feature descriptions
feature_descriptions = [
    {"name": "tid", "description": "Transaction id"},
    {"name": "date_time", "description": "Transaction time"},
    {
        "name": "cc_num",
        "description": "Number of the credit card performing the transaction",
    },
    {"name": "category", "description": "Expense category"},
    {"name": "amount", "description": "Dollar amount of the transaction"},
    {"name": "latitude", "description": "Transaction location latitude"},
    {"name": "longitude", "description": "Transaction location longitude"},
    {"name": "city", "description": "City in which the transaction was made"},
    {"name": "country", "description": "Country in which the transaction was made"},
    {
        "name": "fraud_label",
        "description": "Whether the transaction was fraudulent or not",
    },
    {
        "name": "age_at_transaction",
        "description": "Age of the card holder when the transaction was made",
    },
    {
        "name": "days_until_card_expires",
        "description": "Card validity days left when the transaction was made",
    },
]

for desc in feature_descriptions:
    combined_fg.update_feature_description(desc["name"], desc["description"])